In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
import numpy as np
from torchvision import datasets, transforms


### make dataloader

In [2]:
# load patches data from files
train_images_patches = torch.load('data/mnist77_train.pt')
test_images_patches = torch.load('data/mnist77_test.pt')

# get the label from datasets.MNIST
transform = transforms.Compose([transforms.ToTensor()])
train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform)
train_labels = [label for _, label in train_dataset]
test_labels = [label for _, label in test_dataset]
train_labels = torch.LongTensor(train_labels)
test_labels = torch.LongTensor(test_labels)

# make them to be PyTorch tensors, and dataloader
# train_dataset = torch.utils.data.TensorDataset(train_images_patches[:100], train_labels[:100])
# test_dataset = torch.utils.data.TensorDataset(test_images_patches[:100], test_labels[:100])
train_dataset = torch.utils.data.TensorDataset(train_images_patches, train_labels)
test_dataset = torch.utils.data.TensorDataset(test_images_patches, test_labels)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

# test dataloader
for images, labels in train_loader:
    print(images.shape)
    print(labels.shape)
    break


torch.Size([64, 49, 25])
torch.Size([64])


### define model

In [ ]:
# define a model
# define sub model first
class SubModel(nn.Module):
    def __init__(self, device):
        super(SubModel, self).__init__()
        self.fc0 = nn.Linear(13, 11, bias=False)
        self.fc1 = nn.Linear(11, 9, bias=False)
        self.fc2 = nn.Linear(9, 7, bias=False)
        self.fc3 = nn.Linear(7, 5, bias=False)
        self.fc4 = nn.Linear(5, 3, bias=False)
        self.fc5 = nn.Linear(3, 1, bias=False)
        # masks
        self.mask0 = self.generate_fstmask(13, 11)
        self.mask0 = self.mask0.to(device)
        self.mask1 = self.generate_mask(11, 9)
        self.mask1 = self.mask1.to(device)
        self.mask2 = self.generate_mask(9, 7)
        self.mask2 = self.mask2.to(device)
        self.mask3 = self.generate_mask(7, 5)
        self.mask3 = self.mask3.to(device)
        self.mask4 =self.generate_mask(5, 3)
        self.mask4 = self.mask4.to(device)
        self.mask5 = self.generate_mask(3, 1)
        self.mask5 = self.mask5.to(device)

        self.logg = False
    
    def generate_fstmask(self, in_dim, out_dim):
        assert (in_dim % 2 == 1) and (out_dim % 2 == 1) and (in_dim-out_dim==2), "in_dim and out_dim must be odd and in_dim-out_dim=2"
        mask = torch.zeros(out_dim, in_dim)
        for i in range(out_dim):
            if i % 2 == 0:  # For every second row
                start_index = i  # Determine start index for the 1's in this row
                mask[i, start_index:start_index+3] = 1
        
        return mask
    
    def generate_mask(self, in_dim, out_dim):
        assert (in_dim % 2 == 1) and (out_dim % 2 == 1) and (in_dim-out_dim==2), "main_mid_mask: in_dim and out_dim must be odd and in_dim-out_dim=2"
        mask = torch.zeros(out_dim, in_dim)
        for i in range(out_dim):
            if i % 2 == 0:  # For every second row
                idx_a = max(0, i-2)
                adx_b = min(max(0, i), in_dim-1)
                idx_c = min(max(0, i+1), in_dim-1)
                idx_d = min(max(0, i+2), in_dim-1)
                idx_e = min(max(0, i+4), in_dim-1)
                # print(idx_a, adx_b, idx_c, idx_d, idx_e)
                index_list = [idx_a, adx_b, idx_c, idx_d, idx_e]
                mask[i, index_list] = 1
        
        return mask 
    
    def forward(self, x):
        assert x.shape[1] == 25, "x.shape[1] must be 25, but got {}".format(x.shape[1])
        assert len(x.shape) == 2, "len(x.shape) must be 2, but got {}".format(len(x.shape))

        self.fc0.weight.data *= self.mask0
        x0 = self.fc0(x[:, 0:13]) # x0(b,11)
        x0 = torch.tanh(x0)
        
        # insert x[:, 13:18] to x0[1], x0[3], x0[5], x0[7], x0[9]
        assert torch.sum(x0[:, [1,3,5,7,9]]) == 0, "x0[:, [1,3,5,7,9]] must be 0, but got {}".format(x0[:, [1,3,5,7,9]])
        x0_copy = x0.clone()
        x0_copy[:, [1,3,5,7,9]] += x[:, [13,14,15,16,17]]
        x0 = x0_copy
        self.fc1.weight.data *= self.mask1
        x1 = self.fc1(x0)
        x1 = torch.tanh(x1)

        # insert x[:, 18:22] to x1[1], x1[3], x1[5], x1[7]
        assert torch.sum(x1[:, [1,3,5,7]]) == 0, "x1[:, [1,3,5,7]] must be 0, but got {}".format(x1[:, [1,3,5,7]])
        x1_copy = x1.clone()
        x1_copy[:, [1,3,5,7]] += x[:, [18,19,20,21]]
        x1 = x1_copy
        self.fc2.weight.data *= self.mask2
        x2 = self.fc2(x1)
        x2_copy = x2.clone()
        x2_copy[:,[0,2,4,6]] += x0[:, [2,4,6,8]] # jump connection
        x2 = x2_copy
        x2 = torch.tanh(x2)

        # insert x[:, 22:25] to x2[1], x2[3], x2[5]
        assert torch.sum(x2[:, [1,3,5]]) == 0, "x2[:, [1,3,5]] must be 0, but got {}".format(x2[:, [1,3,5]])
        x2_copy = x2.clone()
        x2_copy[:, [1,3,5]] += x[:, [22,23,24]]
        x2 = x2_copy
        self.fc3.weight.data *= self.mask3
        x3 = self.fc3(x2)
        x3_copy = x3.clone()
        x3_copy[:, [0,2,4]] += x1[:, [2,4,6]] # jump connection
        x3 = x3_copy
        # no insert from now on(fc3)
        x3 = torch.tanh(x3)

        assert torch.sum(x3[:, [1,3]]) == 0, "x3[:, [1,3]] must be 0, but got {}".format(x3[:, [1,3]])
        # no insert from now on(fc4)
        self.fc4.weight.data *= self.mask4
        x4 = self.fc4(x3)
        x4_copy = x4.clone()
        x4_copy[:, [0,2]] += x2[:, [2,4]] # jump connection
        x4 = x4_copy
        x4 = torch.tanh(x4)

        assert torch.sum(x4[:, [1]]) == 0, "x4[:, [1]] must be 0, but got {}".format(x4[:, [1]])
        # no insert from now on(fc5)
        self.fc5.weight.data *= self.mask5
        x5 = self.fc5(x4)
        x5_copy = x5.clone()
        x5_copy[:, [0]] += x3[:, [2]] # jump connection
        x5 = x5_copy
        x5 = torch.tanh(x5)


        if self.logg: print(x0, '\n', x1, '\n',
                            x2, '\n', x3, '\n', x4, '\n', x5)

        return x5

# test model use randn input
# model = SubModel()
# x = torch.randn(1, 25)
# model(x)

# define main model
# main model contain 49 sub models, and concat the 49 submodels' output to a 49-dim vector
class MainModel(nn.Module):
    def __init__(self, device) -> None:
        super(MainModel, self).__init__()
        self.submodels = nn.ModuleList([SubModel(device) for _ in range(49)])
        self.logg = False

        self.fc0 = nn.Linear(27, 33, bias=False)
        self.fc1 = nn.Linear(33, 31, bias=False)
        self.fc2 = nn.Linear(31, 29, bias=False)
        self.fc3 = nn.Linear(29, 27, bias=False)
        self.fc4 = nn.Linear(27, 25, bias=False)
        self.fc5 = nn.Linear(25, 23, bias=False)
        self.fc6 = nn.Linear(23, 21, bias=False)
        self.fc7 = nn.Linear(21, 19, bias=False)

        self.mask0 = self.main_fst_mask(27, 33)
        self.mask0 = self.mask0.to(device)
        self.mask1 = self.main_mid_mask(33, 31)
        self.mask1 = self.mask1.to(device)
        self.mask2 = self.main_mid_mask(31, 29)
        self.mask2 = self.mask2.to(device)
        self.mask3 = self.main_mid_mask(29, 27)
        self.mask3 = self.mask3.to(device)
        self.mask4 = self.main_mid_mask(27, 25)
        self.mask4 = self.mask4.to(device) 
        self.mask5 = self.main_mid_mask(25, 23)
        self.mask5 = self.mask5.to(device)
        self.mask6 = self.main_mid_mask(23, 21)
        self.mask6 = self.mask6.to(device)
        self.mask7 = self.main_mid_mask(21, 19)
        self.mask7 = self.mask7.to(device)

        # or pure fcfc #################################
        # self.fcfc1 = nn.Linear(49, 20, bias=False)
        # self.fcfc2 = nn.Linear(20, 5, bias=False)
        # self.fcfc3 = nn.Linear(5, 5, bias=False)


    def main_fst_mask(self, in_dim, out_dim):
        assert (in_dim % 2 == 1) and (out_dim % 2 == 1) and (out_dim-in_dim==6), "gen_main_fst_mask: in_dim and out_dim must be odd and in_dim-out_dim=6"
        mask = torch.zeros(out_dim, in_dim)
        for i in range(out_dim):
            if i % 2 == 0:  # For every second row
                idx_a = max(0, i-6)
                adx_b = min(max(0, i-4), in_dim-1)
                idx_c = min(max(0, i-3), in_dim-1)
                idx_d = min(max(0, i-2), in_dim-1)
                idx_e = min(i, in_dim-1)
                # print(idx_a, adx_b, idx_c, idx_d, idx_e)
                index_list = [idx_a, adx_b, idx_c, idx_d, idx_e]
                mask[i, index_list] = 1
        
        return mask
    
    def main_mid_mask(self, in_dim, out_dim):
        assert (in_dim % 2 == 1) and (out_dim % 2 == 1) and (in_dim-out_dim==2), "main_mid_mask: in_dim and out_dim must be odd and in_dim-out_dim=2"
        mask = torch.zeros(out_dim, in_dim)
        for i in range(out_dim):
            if i % 2 == 0:  # For every second row
                idx_a = max(0, i-2)
                adx_b = min(max(0, i), in_dim-1)
                idx_c = min(max(0, i+1), in_dim-1)
                idx_d = min(max(0, i+2), in_dim-1)
                idx_e = min(max(0, i+4), in_dim-1)
                # print(idx_a, adx_b, idx_c, idx_d, idx_e)
                index_list = [idx_a, adx_b, idx_c, idx_d, idx_e]
                mask[i, index_list] = 1
        
        return mask 
    
    
    def forward(self, xx):
        assert len(xx.shape) == 3, "Main. len(x.shape) must be 3, but got {}".format(len(xx.shape))
        assert (xx.shape[1]==49) and (xx.shape[2]==25), "Main. x.shape[1] and x.shape[2] must be 25, but got {} and {}".format(xx.shape[1], xx.shape[2])
        # the 25 input to 25 submodels
        sub_results = []
        for i in range(49):
            sub_results.append(self.submodels[i](xx[:, i, :]))
        
        sub_results = torch.cat(sub_results, dim=1)
        if self.logg: print('sub_results:',sub_results.shape)
        assert sub_results.shape[1] == 49, "sub_results.shape[1] must be 49, but got {}".format(sub_results.shape[1])

        # # directly use fcfc to process the 25-dim vector
        # x1c = self.fcfc1(sub_results)
        # x1c = torch.tanh(x1c)
        # x2 = self.fcfc2(x1c)
        # x2 = torch.tanh(x2)
        # x3 = self.fcfc3(x2)
        # x3 = torch.tanh(x3)
        # output = torch.cat([x2, x3], dim=1) # (b, 10)
        ############################################################

        # process the 49-dim vector
        self.fc0.weight.data *= self.mask0
        x0 = self.fc0(sub_results[:, 0:27]) # (b, 33)
        x0 = torch.tanh(x0) # (b, 33)

        assert torch.sum(x0[:, [7,9,11,13,15,17,19,21,23,25]]) == 0, "x0[:, [7,9,11,13,15,17,19,21,23,25]] must be 0, but got {}".format(x0[:, [7,9,11,13,15,17,19,21,23,25]])
        x0_copy = x0.clone()
        x0_copy[:, [7,9,11,13,15,17,19,21,23,25]] += sub_results[:, [27,28,29,30,31,32,33,34,35,36]]
        x0 = x0_copy
        # x0c = torch.cat([ x0[:, 0:7], sub_results[:, 27:28], x0[:, 8:9], sub_results[:, 28:29], x0[:, 10:11], sub_results[:, 29:30], 
        #                  x0[:, 12:13], sub_results[:, 30:31], x0[:, 14:15], sub_results[:, 31:32], x0[:, 16:17], sub_results[:, 32:33], 
        #                  x0[:, 18:19], sub_results[:, 33:34], x0[:, 20:21], sub_results[:, 34:35], x0[:, 22:23], sub_results[:, 35:36], 
        #                  x0[:, 24:25], sub_results[:, 36:37], x0[:, 26:]], dim=1) # (b, 33)
        self.fc1.weight.data *= self.mask1
        x1 = self.fc1(x0)
        x1 = torch.tanh(x1) # (b, 31)

        assert torch.sum(x1[:, [9,11,13,15,17,19,21]]) == 0, "x1[:, [9,11,13,15,17,19,21]] must be 0, but got {}".format(x1[:, [9,11,13,15,17,19,21]])
        x1_copy = x1.clone()
        x1_copy[:, [9,11,13,15,17,19,21]] += sub_results[:, [37,38,39,40,41,42,43]]
        x1 = x1_copy
        # x1c = torch.cat([ x1[:, 0:9], sub_results[:, 37:38], x1[:, 10:11], sub_results[:, 38:39], x1[:, 12:13], sub_results[:, 39:40],
        #                  x1[:, 14:15], sub_results[:, 40:41], x1[:, 16:17], sub_results[:, 41:42], x1[:, 18:19], sub_results[:, 42:43],
        #                  x1[:, 20:21], sub_results[:, 43:44], x1[:, 22:]], dim=1) # (b, 31)
        self.fc2.weight.data *= self.mask2
        x2 = self.fc2(x1)
        x2_copy = x2.clone()
        x2_copy[:, [0,2,4,6,8,10,12,14,16,18,20,22,24,26,28]] += x0[:,[2,4,6,8,10,12,14,16,18,20,22,24,26,28,30]] # jump connection
        x2 = x2_copy
        x2 = torch.tanh(x2) # (b, 29)

        assert torch.sum(x2[:, [11,13,15,17]]) == 0, "x2[:, [11,13,15,17]] must be 0, but got {}".format(x2[:, [11,13,15,17]])
        x2_copy = x2.clone()
        x2_copy[:, [11,13,15,17]] += sub_results[:, [44,45,46,47]]
        x2 = x2_copy
        # x2c = torch.cat([ x2[:, 0:11], sub_results[:, 44:45], x2[:, 12:13], sub_results[:, 45:46], x2[:, 14:15], sub_results[:, 46:47],
        #                 x2[:, 16:17], sub_results[:, 47:48], x2[:, 18:]], dim=1) # (b, 29)
        self.fc3.weight.data *= self.mask3
        x3 = self.fc3(x2)
        x3_copy = x3.clone()
        x3_copy[:, [0,2,4,6,8,10,12,14,16,18,20,22,24,26]] += x1[:, [2,4,6,8,10,12,14,16,18,20,22,24,26,28]] # jump connection
        x3 = x3_copy
        x3 = torch.tanh(x3) # (b, 27)

        assert torch.sum(x3[:, [13]]) == 0, "x3[:, [13]] must be 0, but got {}".format(x3[:, [13]])
        x3_copy = x3.clone()
        x3_copy[:, [13]] += sub_results[:, [48]]
        x3 = x3_copy
        self.fc4.weight.data *= self.mask4
        x4 = self.fc4(x3)
        x4_copy = x4.clone()
        x4_copy[:, [0,2,4,6,8,10,12,14,16,18,20,22,24]] += x2[:, [2,4,6,8,10,12,14,16,18,20,22,24,26]] # jump connection
        x4 = x4_copy
        x4 = torch.tanh(x4) # (b, 25)

        self.fc5.weight.data *= self.mask5
        x5 = self.fc5(x4)
        x5_copy = x5.clone()
        x5_copy[:, [0,2,4,6,8,10,12,14,16,18,20,22]] += x3[:, [2,4,6,8,10,12,14,16,18,20,22,24]] # jump connection
        x5 = x5_copy
        x5 = torch.tanh(x5) # (b, 23)

        self.fc6.weight.data *= self.mask6
        x6 = self.fc6(x5)
        x6_copy = x6.clone()
        x6_copy[:, [0,2,4,6,8,10,12,14,16,18,20]] += x4[:, [2,4,6,8,10,12,14,16,18,20,22]] # jump connection
        x6 = x6_copy
        x6 = torch.tanh(x6) # (b, 21)

        self.fc7.weight.data *= self.mask7
        x7 = self.fc7(x6)
        x7_copy = x7.clone()
        x7_copy[:, [0,2,4,6,8,10,12,14,16,18]] += x5[:, [2,4,6,8,10,12,14,16,18,20]] # jump connection
        x7 = x7_copy
        x7 = torch.tanh(x7) # (b, 19)

        output = x7[:, [0,2,4,6,8,10,12,14,16,18]] # (b, 10)
        

        return output

# use randn input to test main model
# model = MainModel()
# x = torch.randn(1, 49, 25)
# model(x) # (1, 10)

# train the model ********************************************************
device = torch.device('cpu')
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = MainModel(device).to(device)
model.load_state_dict(torch.load('data/dianet_patch_mnist_49_25.pth'))
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)
# torch.autograd.set_detect_anomaly(True)

# train
model.train()
max_acc, min_loss = 0.9526, 0.82
for epoch in range(200):
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        images = images.view(-1, 49, 25)
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print('epoch {}, loss {}'.format(epoch, loss.item()))

    # test model
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            images = images.view(-1, 49, 25)
            outputs = model(images)
            loss = criterion(outputs, labels)
            _, predicted = torch.max(outputs.data, dim=1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        
        # save best model, both accuracy and loss
        if correct/total > max_acc: #and loss.item() < min_loss:
            max_acc = correct/total
            min_loss = loss.item()
            torch.save(model.state_dict(), 'data/dianet_patch_mnist_49_25.pth') # 0.9526
            print('saved at epoch {}, acc {}, loss {}'.format(epoch, max_acc, min_loss))
        

In [7]:
print(model)

MainModel(
  (submodels): ModuleList(
    (0): SubModel(
      (fc0): Linear(in_features=13, out_features=11, bias=False)
      (fc1): Linear(in_features=11, out_features=9, bias=False)
      (fc2): Linear(in_features=9, out_features=7, bias=False)
      (fc3): Linear(in_features=7, out_features=5, bias=False)
      (fc4): Linear(in_features=5, out_features=3, bias=False)
      (fc5): Linear(in_features=3, out_features=1, bias=False)
    )
    (1): SubModel(
      (fc0): Linear(in_features=13, out_features=11, bias=False)
      (fc1): Linear(in_features=11, out_features=9, bias=False)
      (fc2): Linear(in_features=9, out_features=7, bias=False)
      (fc3): Linear(in_features=7, out_features=5, bias=False)
      (fc4): Linear(in_features=5, out_features=3, bias=False)
      (fc5): Linear(in_features=3, out_features=1, bias=False)
    )
    (2): SubModel(
      (fc0): Linear(in_features=13, out_features=11, bias=False)
      (fc1): Linear(in_features=11, out_features=9, bias=False)
  

## bp test

In [16]:
# Assume x0 and x1 are the outputs of previous layers
x0 = torch.randn(10, 33, requires_grad=True)  # batch size 10, feature size 33
x1 = torch.randn(10, 31, requires_grad=True)  # batch size 10, feature size 31

# Create a copy of x1
x1_copy = x1.clone()

# Perform the addition operation on the copy
x1_copy[:, [0,2,4,6,8]] += x0[:, [2,4,6,8,10]]

# Replace x1 with the modified copy
x1 = x1_copy

# Compute a dummy loss and backpropagate
loss = x1.sum()
loss.backward()

# Print gradients
print("Gradient for x0:\n", x0.grad)
print("Gradient for x1:\n", x1.grad)

Gradient for x0:
 tensor([[0., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0.,

C:\Users\comparch\AppData\Local\Temp\ipykernel_21328\3330320129.py:22: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at C:\cb\pytorch_1000000000000\work\build\aten\src\ATen/core/TensorBody.h:485.)
  print("Gradient for x1:\n", x1.grad)


In [10]:
# write some rules
# matrix (9,11)
'''
rowid:colid has 1, others are 0
0:0,1,2,4
2:0,2,3,4,6
4:2,4,5,6,7
6:4,6,7,8,10
8:6,8,9,10
'''
import torch

def generate_mask(in_dim, out_dim):
    assert (in_dim % 2 == 1) and (out_dim % 2 == 1) and (in_dim-out_dim==2), "in_dim and out_dim must be odd and in_dim-out_dim=2"
    mask = torch.zeros(out_dim, in_dim)
    for i in range(out_dim):
        if i % 2 == 0:  # For every second row
            start_index = i  # Determine start index for the 1's in this row
            mask[i, start_index:start_index+3] = 1
            if i >= 2 and i <= out_dim-3: 
                mask[i, start_index-2] = 1
                mask[i, start_index+3+1] = 1
    
    return mask

# test generate_mask
mask = generate_mask(11, 9)
print(mask)
    
    

tensor([[1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 1., 1., 1., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 1., 1., 1., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 1., 1., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1.]])


In [22]:
def main_fst_mask(in_dim, out_dim):
    assert (in_dim % 2 == 1) and (out_dim % 2 == 1) and (out_dim-in_dim==6), "in_dim and out_dim must be odd and in_dim-out_dim=2"
    mask = torch.zeros(out_dim, in_dim)
    for i in range(out_dim):
        if i % 2 == 0:  # For every second row
            idx_a = max(0, i-6)
            adx_b = min(max(0, i-4), in_dim-1)
            idx_c = min(max(0, i-3), in_dim-1)
            idx_d = min(max(0, i-2), in_dim-1)
            idx_e = min(i, in_dim-1)
            print(idx_a, adx_b, idx_c, idx_d, idx_e)
            index_list = [idx_a, adx_b, idx_c, idx_d, idx_e]
            mask[i, index_list] = 1
    
    return mask

# test generate_mask
mask = main_fst_mask(27, 33)
print(mask)

0 0 0 0 0
0 0 0 0 2
0 0 1 2 4
0 2 3 4 6
2 4 5 6 8
4 6 7 8 10
6 8 9 10 12
8 10 11 12 14
10 12 13 14 16
12 14 15 16 18
14 16 17 18 20
16 18 19 20 22
18 20 21 22 24
20 22 23 24 26
22 24 25 26 26
24 26 26 26 26
26 26 26 26 26
tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0.,

In [25]:
def main_mid_mask(in_dim, out_dim):
    assert (in_dim % 2 == 1) and (out_dim % 2 == 1) and (in_dim-out_dim==2), "main_mid_mask: in_dim and out_dim must be odd and in_dim-out_dim=2"
    mask = torch.zeros(out_dim, in_dim)
    for i in range(out_dim):
        if i % 2 == 0:  # For every second row
            idx_a = max(0, i-2)
            adx_b = min(max(0, i), in_dim-1)
            idx_c = min(max(0, i+1), in_dim-1)
            idx_d = min(max(0, i+2), in_dim-1)
            idx_e = min(max(0, i+4), in_dim-1)
            print(idx_a, adx_b, idx_c, idx_d, idx_e)
            index_list = [idx_a, adx_b, idx_c, idx_d, idx_e]
            mask[i, index_list] = 1
    
    return mask

# test generate_mask
mask = main_mid_mask(9, 7)
print(mask)

0 0 1 2 4
0 2 3 4 6
2 4 5 6 8
4 6 7 8 8
tensor([[1., 1., 1., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 1., 1., 1., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 1., 1., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 1., 1., 1.]])


In [11]:
import numpy as np

def generate_mask(input_dim, output_dim):
    mask = np.zeros((output_dim, input_dim))  # Create an array of zeros

    for i in range(output_dim):
        if i % 2 == 0:  # For every second row
            start_index = i  # Determine start index for the 1's in this row
            mask[i, start_index:start_index+3] = 1

    return mask

# Test the function with linear(7,5)
mask = generate_mask(13, 11)
print(mask)


[[1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1.]]
